In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import pyplot
from sklearn.model_selection import train_test_split 
from sklearn.metrics import make_scorer, accuracy_score
from sklearn import preprocessing
import random



In [2]:
def De_Algorithm(fobj, bounds, mut=0.8, crossp=0.7, popsize=100, its=1000):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossp
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = fobj(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
                    
        print("Iteration number= %s" % (i))
        print("Best Fitness= %s" % (fitness[best_idx]))
        print("Best values= %s" % (best))
        yield best, fitness[best_idx]

## Data-set of your choice
Let's use the TalkingData set which is available on Kaggle.

In [3]:
#address_train = '../input/talkingdata-adtracking-fraud-detection/train_sample.csv'
address_train = "C:/Users/DASLAB Hareland 3/Desktop/data Fraud/df_train.csv"

### Some handy functions

In [4]:
def Drop_cols(df, x):
    df.drop(labels = x, axis = 1, inplace = True)
    return df

def Plot_Hist_column(df, x):
    pyplot.hist(df[x], log = True)
    pyplot.title(x)
    pyplot.show()
    
def Plot_Hist_columns(df, xlist):
    [Plot_Hist_column(df, x) for x in xlist]  
    pyplot.show()
    
def Make_X_Y(df):
    Y = pd.DataFrame()
    Y['is_attributed'] = df['is_attributed']
    X = df.copy()
    X.drop(labels = ["is_attributed"], axis = 1, inplace = True)
    return X, Y

def Train_Test_training_valid(X, Y, ratio):
    Num_of_line = 100
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=ratio)
    X_training, X_valid, y_training, y_valid = \
    train_test_split(X_train, y_train, test_size=ratio, random_state=0)
    return X_training, y_training, X_valid, y_valid

def read_train_test_data_balanced(address_train):
    
    #Read Training data, all class 1 and add same amount 0
    iter_csv = pd.read_csv(address_train, iterator=True, chunksize=10000000, parse_dates=['click_time'])
    df_train_1 = pd.concat([chunk[chunk['is_attributed'] > 0] for chunk in iter_csv])
    iter_csv = pd.read_csv(address_train, iterator=True, chunksize=10000000, parse_dates=['click_time'], nrows=2000000)
    df_train_0 = pd.concat([chunk[chunk['is_attributed'] == 0] for chunk in iter_csv])
    #seperate same number values as train data with class 1
    df_train_0 = df_train_0.head(len(df_train_1))
    #Merge 0 and 1 data
    df_train = Merge_data(df_train_1, df_train_0)
    return df_train

def read_train_data(address_train):
    
    n = 10000000 #number of records in file
    s = 1000000 #desired sample size
    skip = sorted(random.sample(range(n),n-s))
    df_train = pd.read_csv(address_train, skiprows=skip)
    #df_train = pd.read_csv(address_train)
    
    return df_train

def Merge_data(df1, df2):
    frames = [df1, df2]
    df = pd.concat(frames)
    return df

def Normalized(df):
    df_col_names = df.columns
    x = df.values 
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    df.columns = df_col_names
    return df

## Preparing a dataset
read data balanced. Read all 1 values from the train data set and then add the same number of 0 and keep it. Now we do have a balance data set with an equal number of 0 and 1.

In [5]:
df_train = read_train_data(address_train)
df_train.head(3)

NameError: name 'pandas' is not defined

In [0]:
df_train.shape

In [0]:
# See the output paramters distribution 
xlist = ['is_attributed']
Plot_Hist_columns(df_train, xlist)

In [0]:
# Lets drop click_time and attributed_time for sack of simplicity 
df_train.drop(labels = ["click_time", "attributed_time"], axis = 1, inplace = True)

In [0]:
df_train.head()

In [0]:
df_train = Normalized(df_train)
df_train.head()

In [0]:
df_train.shape

In [0]:
#Define an objective funtion.
def Objective_Function(w):
    #w = w.tolist()
    
    '''diff = abs(round(w[0]*(df_train['ip']**w[1])*(df_train['app']**w[2])*(df_train['device']**w[3])*\
    (df_train['os']**w[4])*(df_train['channel']**w[5]) + w[6]) - df_train['is_attributed'])
    return sum(diff)*100/len(df_train)'''



    a = (w[0]*(df_train['ip']**w[1])*(df_train['app']**w[2])*(df_train['device']**w[3])*\
    (df_train['os']**w[4])*(df_train['channel']**w[5]) + w[6])

    b = w[7]*(df_train['ip']**w[8])+w[9]*(df_train['app']**w[10])*(df_train['os']**w[11])

    c = w[12]*(df_train['device']**w[13]) + w[14]*(df_train['channel']**w[15])

    d = w[16]*(df_train['ip']**w[17])+w[18]*(df_train['device']**w[19])


    Z = ((a / b) + c/d)
    Z = [0 if z < w[20] else 1 for z in Z]


    diff = abs(Z - df_train['is_attributed'])
    return sum(diff)*100/len(df_train)





In [0]:
def corr(w):
    #w = w.tolist()
    
    '''a = round(w[0]*(df_train['ip']**w[1])*(df_train['app']**w[2])*(df_train['device']**w[3])*\
    (df_train['os']**w[4])*(df_train['channel']**w[5]) + w[6])'''

    a = (w[0]*(df_train['ip']**w[1])*(df_train['app']**w[2])*(df_train['device']**w[3])*\
    (df_train['os']**w[4])*(df_train['channel']**w[5]) + w[6])

    b = w[7]*(df_train['ip']**w[8])+w[9]*(df_train['app']**w[10])*(df_train['os']**w[11])

    c = w[12]*(df_train['device']**w[13]) + w[14]*(df_train['channel']**w[15])
    d = w[16]*(df_train['ip']**w[17])+w[18]*(df_train['device']**w[19])


    Z = ((a / b) + c/d)
    Z = [0 if z < w[20] else 1 for z in Z]
    
    return Z

In [0]:
'''w = [1,1,1,1,\
    1,1,1,1,\
    1,1,1,1,\
    1,1,1,1,\
    1]
w = [w*0.5 for w in w]
print(w)
Objective_Function(w)'''

In [0]:
upper = 5
lower = -5

#Run the DE algorithm on objective function in your favorite range of hyperparameters.
result = list(De_Algorithm(Objective_Function,
                 [(-100, 100), (lower, upper),(lower, upper),(lower, upper),(lower, upper),(lower, upper),(-1000, 1000),\
                  (lower, upper), (lower, upper),(lower, upper),(lower, upper),(lower, upper),\
                  (lower, upper), (lower, upper),(lower, upper),(lower, upper),
                  (lower, upper), (lower, upper),(lower, upper),(lower, upper),
                  (-1e15,1e11)],\
                  mut=0.7, crossp=0.8, popsize=100, its=5000))

In [0]:
df = pd.DataFrame(result)
df.head()

In [0]:
df = pd.DataFrame(result)
# seperate the best of hyperparamters.
def Best_coffs(df):
    
    #df['w1'], df['w2'], df['w3'], df['w4'], df['w5'], df['w6'], df['w7']  = zip(*df[0]) # Unzip
    #df['w1'], df['w2'], df['w3'], df['w4'], df['w5'], df['w6'], df['w7'], df['w8'], df['w9'], df['w10'], df['w11'], df['w12'], df['w13'], df['w14'], df['w15'], df['w16'], df['w17'], ['w18'], df['w19'], df['w20'], df['w21'] = zip(*df[0]) # Unzip]
    df['w1'], df['w2'], df['w3'], df['w4'], df['w5'], df['w6'], df['w7'], df['w8'], df['w9'], df['w10'], df['w11'], df['w12'], df['w13'], df['w14'], df['w15'], df['w16'], df['w17'], df['w18'], df['w19'], df['w20'], df['w21']  = zip(*df[0]) # Unzip  = zip(*df[0]) # Unzip

    cols = [0] # Drop the first column
    df.drop(df.columns[cols],axis = 1,inplace = True) # Drop the first column
    df.columns.values[0] = "Fitness" # name the first column as Fitness
    best_coff = df.iloc[len(df)-1,1:] # insert the best coefficients into the best_coff
    return best_coff
Best_coffs(df)

## Visualization of searching progress

In [0]:
def Plot_DEA_Evolution(df):
    
    data_ncol=len(df.columns) # number of paramters 
    fig = plt.figure(figsize=(15,15)) # you may change these to change the distance between plots.

    for i in range(1,(data_ncol+1)):
        if i<data_ncol:
            plt.subplot(5, 5, i)
            plt.plot(df['w{}'.format(i)],'bo', markersize=4)
            plt.xlabel('Iteration')
            plt.ylabel('w{}'.format(i))
            plt.grid(True)
        else:       
            plt.subplot(5, 5, data_ncol)
            plt.plot(df['Fitness'],'red', markersize=4)
            plt.xlabel('Iteration')
            plt.ylabel('Fitness')
            #plt.ylim(0,100)
            plt.grid(True)
    plt.show()

    
Plot_DEA_Evolution(df)

In [0]:
df = pd.DataFrame(result)
w = Best_coffs(df)
a = corr(w)
df_train['corr'] = a
df_train.head(15)